#blueqatのバックエンドを作る（簡易編）
今回はblueqatのバックエンドをqasmをベースに作る方法を確認します。今回はqiskitとcirqバックエンドを実装します。IBM社のQiskitとGoogle非公式のCirqをバックエンドとして利用してみます。

まずはインストールです。

In [0]:
pip install blueqat qiskit cirq

##まずQiskit
まずはQiskitです。ツールを読み込み、引数を設定してバックエンドが呼び出された時に返す値を設定すれば終わります。

In [0]:
import warnings
from collections import Counter
from blueqat.backends.qasm_parser_backend_generator import generate_backend

def _qasm_runner_qiskit(qasm, qiskit_backend=None, shots=None, returns=None, **kwargs):
    if returns is None:
        returns = "shots"
    elif returns not in ("shots", "draw", "_exception",
                         "qiskit_circuit", "qiskit_job", "qiskit_result"):
        raise ValueError("`returns` shall be None, 'shots', 'draw', " +
                         "'qiskit_circuit', 'qiskit_job', 'qiskit_result' or '_exception'")

    import_error = None
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            from qiskit import Aer, QuantumCircuit, execute
    except Exception as e:
        import_error = e

    if import_error:
        if returns == "_exception":
            return e
        if isinstance(import_error, ImportError):
            raise ImportError("Cannot import qiskit. To use this backend, please install qiskit." +
                              " `pip install qiskit`.")
        else:
            raise ValueError("Unknown error raised when importing qiskit. To get exception, " +
                             'run this backend with arg `returns="_exception"`')
    else:
        if returns == "_exception":
            return None
        qk_circuit = QuantumCircuit.from_qasm_str(qasm)
        if returns == "qiskit_circuit":
            return qk_circuit
        if returns == "draw":
            return qk_circuit.draw(**kwargs)
        if shots is None:
            shots = 1024
        if qiskit_backend is None:
            qiskit_backend = Aer.get_backend("qasm_simulator")
        job = execute(qk_circuit, backend=qiskit_backend, shots=shots, **kwargs)
        if returns == "qiskit_job":
            return job
        result = job.result()
        if returns == "qiskit_result":
            return result
        counts = Counter({bits[::-1]: val for bits, val in result.get_counts().items()})
        return counts

ibmq_backend = generate_backend(_qasm_runner_qiskit)

そして、最後にbackendを登録して終わりです。

In [0]:
from blueqat import BlueqatGlobalSetting
BlueqatGlobalSetting.register_backend("myibmq", ibmq_backend)

これで利用できるようになりました。デフォルトでは、計算結果のサンプルを返すので、重ね合わせ回路を実行してみます。普通にblueqatを実行してbackend指定するだけでできます。

In [4]:
from blueqat import Circuit
Circuit().h[0].m[:].run(backend="myibmq")

Counter({'0': 527, '1': 497})

無事動きました。簡単ですね。次に回路を描く機能を使ってみます。

```python
        if returns == "draw":
            return qk_circuit.draw(**kwargs)
```

これにより回路の描画も実装できます。

In [6]:
Circuit().h[0].m[:].run(backend="myibmq",returns="draw")

┌───┐┌─┐
q_0: |0>┤ H ├┤M├
        └───┘└╥┘
 c_0: 0 ══════╩═

無事できました。

##Cirqを実装
こちらは許可を得て、こちらの記事より掲載させていただきます。

Cirqバックエンドを作ったSJSYさんの記事  
https://sjsy.hatenablog.com/entry/20191212/1576158640

In [0]:
def _qasm_runner_cirq(qasm, shots=None, returns=None, **kwargs):
    if returns is None:
        returns = "cirq_result"
    elif returns not in ("cirq_result", 'draw'):
        raise ValueError("`returns` shall be None, 'draw', 'cirq_result' or '_exception'")

    import_error = None
    try:
        with warnings.catch_warnings():
            from cirq import Circuit, Simulator
            from cirq.contrib.qasm_import import circuit_from_qasm
            import cirq
    except Exception as e:
        import_error = e

    if import_error:
        if returns == "_exception":
            return e
        if isinstance(import_error, ImportError):
            raise ImportError("Cannot import Cirq. To use this backend, please install qiskit." +
                              " `pip install Cirq`.")
        else:
            raise ValueError("Unknown error raised when importing Cirq. To get exception, " +
                             'run this backend with arg `returns="_exception"`')
    else:
        if returns == "_exception":
            return None
        cirq_circuit = circuit_from_qasm(qasm)
        if returns == "draw":
            return cirq_circuit
        if shots is None:
            shots = 1024
        simulator = cirq.Simulator()
        result = simulator.run(cirq_circuit, repetitions=shots, **kwargs)
        if returns == "cirq_result":
            return result
        return result

cirq_backend = generate_backend(_qasm_runner_cirq)

BlueqatGlobalSetting.register_backend("mycirq", cirq_backend)

基本的にはQiskitの場合と同じです。回路の描画を試してみます。

In [9]:
Circuit().h[0].m[:].run(backend="mycirq",returns="draw")

q_0: ───H───M('c_0')───

無事に登録ができました。基本的なステップは上記コードを参照すればあらゆるバックエンドを統合できます。